In [2]:
import urllib.parse
import base64

def parse_sesame_uri(uri_string: str):
    """
    解析从 Sesame App 分享的 ssm:// 链接，提取 device_secret 和 app_private_key。
    """
    try:
        parsed_uri = urllib.parse.urlparse(uri_string)
        params = urllib.parse.parse_qs(parsed_uri.query)

        if 'sk' not in params:
            print("错误：链接中未找到 'sk' 参数。")
            return

        # 1. 从URL中提取 'sk' 的值
        sk_b64_url_encoded = params['sk'][0]
        print(f"[*] 提取到的 'sk' 值: {sk_b64_url_encoded[:10]}...")

        # 2. Base64 解码
        #    URL安全Base64常用 '-' 和 '_' 代替 '+' 和 '/'，这里进行替换
        #    并补充必要的'=' padding以保证长度是4的倍数
        sk_b64 = sk_b64_url_encoded.replace('-', '+').replace('_', '/')
        padding = '=' * (4 - len(sk_b64) % 4)
        
        print("[*] 正在进行 Base64 解码...")
        decoded_sk = base64.b64decode(sk_b64 + padding)
        print(f"[*] 解码成功，总长度: {len(decoded_sk)} 字节")

        # 3. 校验长度是否为预期的49字节
        if len(decoded_sk) != 49:
            print("\n错误：解码后的密钥数据长度不正确！")
            print(f"预期长度: 49 字节")
            print(f"实际长度: {len(decoded_sk)} 字节")
            print("请确认您已从手机App中复制了【完整】的 ssm:// 链接。")
            return

        # 4. 根据结构切割出两个密钥
        key_index = decoded_sk[0]
        device_secret = decoded_sk[1:17]
        app_private_key = decoded_sk[17:49]

        # 5. 打印结果
        print("\n" + "="*40)
        print("          密钥解析成功！")
        print("="*40)
        print(f"密钥编号 (Key Index): {key_index}")
        print("\n请将下面的值复制到您的主控制脚本中：")
        print("-" * 40)
        print(f"设备共享密码 (DEVICE_SECRET_HEX):")
        print(device_secret.hex())
        print("-" * 40)
        print(f"App私钥 (APP_PRIVATE_KEY_HEX):")
        print(app_private_key.hex())
        print("="*40)

    except Exception as e:
        print(f"\n解析过程中发生错误: {e}")
        print("请检查您的链接是否正确且完整。")

if __name__ == "__main__":
    # 提示用户输入链接
    full_uri = input("请从手机App分享钥匙，然后将完整的 ssm:// 链接粘贴到此处:\n")
    if full_uri:
        parse_sesame_uri(full_uri)

[*] 提取到的 'sk' 值: BUuyc5Natg...
[*] 正在进行 Base64 解码...
[*] 解码成功，总长度: 39 字节

错误：解码后的密钥数据长度不正确！
预期长度: 49 字节
实际长度: 39 字节
请确认您已从手机App中复制了【完整】的 ssm:// 链接。
